In [1]:
# pip install nltk

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
from fractions import Fraction
import re
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
import numpy as np

In [3]:
# x1 = "sunshine state enjoy sunshine"
# x2 = "brown fox jump high, brown fox run"
# x3 = "sunshine state fox run fast"
x1 =  "sunshine state enjoy sunshine".lower()
x2 = "rown fox jump high brown fox run".lower()
x3 = "sunshine state fox run fast".lower()

x2 = re.sub(r"[,]","",x2)
corp = [x1,x2,x3]
df1 = pd.DataFrame(corp)
df1

,0
0,sunshine state enjoy sunshine
1,rown fox jump high brown fox run
2,sunshine state fox run fast


In [4]:
vec = CountVectorizer()
X = vec.fit_transform(corp)

# Vocabolary

In [5]:
features = vec.get_feature_names_out()
features

array(['brown', 'enjoy', 'fast', 'fox', 'high', 'jump', 'rown', 'run',
       'state', 'sunshine'], dtype=object)

# Bag of Words

In [6]:
print(X.toarray())

[[0 1 0 0 0 0 0 0 1 2]
 [1 0 0 2 1 1 1 1 0 0]
 [0 0 1 1 0 0 0 1 1 1]]


In [7]:
df2 = pd.DataFrame(X.toarray(), columns=vec.get_feature_names_out(), index=["x1","x2","x3"])
df2

,brown,enjoy,fast,fox,high,jump,rown,run,state,sunshine
s1,0,1,0,0,0,0,0,0,1,2
s2,1,0,0,2,1,1,1,1,0,0
s3,0,0,1,1,0,0,0,1,1,1


# TF

In [8]:
df2.loc["x1"] = [Fraction(i,len(x1.split())) for i in df2.loc["s1"]]
df2.loc["x2"] = [Fraction(i,len(x2.split())) for i in df2.loc["s2"]]
df2.loc["x3"] = [Fraction(i,len(x3.split())) for i in df2.loc["s3"]]

In [9]:
df2

,brown,enjoy,fast,fox,high,jump,rown,run,state,sunshine
s1,0,1/4,0,0,0,0,0,0,1/4,1/2
s2,1/7,0,0,2/7,1/7,1/7,1/7,1/7,0,0
s3,0,0,1/5,1/5,0,0,0,1/5,1/5,1/5


# IDF

In [10]:
total_docs = len(df2)
idf = dict()

for i in range(len(features)):
  value = 0
  if features[i] in x1.split():
    value+=1
  if features[i] in x2.split():
    value+=1
  if features[i] in x3.split():
    value+=1
  idf[features[i]]= math.log10(Fraction(total_docs,value))

In [11]:
pd.DataFrame(idf,index=["IDF values"])

,brown,enjoy,fast,fox,high,jump,rown,run,state,sunshine
IDF values,0.477121,0.477121,0.477121,0.176091,0.477121,0.477121,0.477121,0.176091,0.176091,0.176091


# TF-IDF

In [12]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)

In [13]:
tfidf_vector = tfidf_vectorizer.fit_transform(corp)

In [14]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=["x1","x2","x3"], columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df

,brown,enjoy,fast,fox,high,jump,rown,run,state,sunshine
s1,0.000000,0.50689,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.385503,0.771006
s2,0.380914,0.00000,0.000000,0.579391,0.380914,0.380914,0.380914,0.289695,0.000000,0.000000
s3,0.000000,0.00000,0.549351,0.417796,0.000000,0.000000,0.000000,0.417796,0.417796,0.417796


# Cosine Similarity